In [3]:
!pip install pyaspeller
!pip install deep_translator 
!pip install pymorphy2
!pip install pymystem3

from functools import lru_cache
from tqdm import tqdm
import numpy as np
import pandas as pd

from multiprocessing import Pool
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer

In [4]:
STOPWORDS = stopwords.words('russian')
RANDOM_STATE = np.random.seed(42)

In [5]:
m = Mystem()

Installing mystem to /Users/aleksandrakozevnikova/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [6]:
train = pd.read_csv('train_ml.csv')
val = pd.read_csv('new_test_ml.csv')

In [7]:
train =train.drop_duplicates()

In [8]:
train

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,28.06.2019 13:54
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,15.07.2020 14:54
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,04.08.2020 6:38
...,...,...,...,...
74995,trust,Три года длится эпопея с закрытием кредитной к...,1.0,26.10.2016 11:03
74996,fk_otkritie,"Являлся клиентом Бинбанка, пока после слияния ...",2.0,01.02.2020 21:07
74997,sberbank,8 мая 2016 г. мною вносились деньги на банковс...,1.0,14.06.2016 16:03
74998,uralsib,Здравствуйте! Сегодня 21.04.2020 захожу в моби...,3.0,21.04.2020 8:24


In [9]:
val

,Unnamed: 0,bank,feeds,date
0,0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53
1,1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19
2,2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46
3,3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48
4,4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41
...,...,...,...,...
17215,17215,sberbank,"21.09.2018 в 15:26:56, я, О-ва К. К., снимала ...",22.09.2018 16:08
17216,17216,homecreditbank,"Наверняка не засчитают, но тем не менее. Нужна...",06.10.2011 9:35
17217,17217,interprombank,"15.08.2019 я, как физ.лицо обратилась в офис н...",21.08.2019 14:13
17218,17218,otpbank,"У банка есть рекламный продукт-игра ""Мои рыбки...",29.05.2020 16:12


In [10]:
train = train[['feeds', 'grades']]
val = val[['feeds']]

In [11]:
@lru_cache(maxsize=2048)
def lemmatize_word(token, mystem=Mystem()):
    return " ".join(mystem.lemmatize(token))

In [12]:
train["feeds"] = train.apply(lambda x: x["feeds"].lower(), axis=1)
val["feeds"] = val.apply(lambda x: x["feeds"].lower(), axis=1)
X_test = val[['feeds']]

<ipython-input-12-434434c9bd69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["feeds"] = train.apply(lambda x: x["feeds"].lower(), axis=1)
<ipython-input-12-434434c9bd69>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val["feeds"] = val.apply(lambda x: x["feeds"].lower(), axis=1)


In [13]:
X_train, y_train = train[["feeds"]], train[["grades"]]

In [14]:
X_test = val

In [15]:
def remove_stopwords(lemmas):
    return " ".join([l for l in lemmas.split() if l not in STOPWORDS])

In [19]:
X_train['feeds'] = list(map(remove_stopwords, X_train['feeds']))
X_test['feeds'] = list(map(remove_stopwords, X_test['feeds']))

<ipython-input-19-5828b27fe8af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['feeds'] = list(map(remove_stopwords, X_train['feeds']))
<ipython-input-19-5828b27fe8af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['feeds'] = list(map(remove_stopwords, X_test['feeds']))


In [20]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)

bow = vec.fit_transform(X_train['feeds'])
bow_test = vec.transform(X_test['feeds'])

In [21]:
bow = pd.DataFrame.sparse.from_spmatrix(bow)
bow_test = pd.DataFrame.sparse.from_spmatrix(bow_test)

In [22]:
bow.shape

(75000, 357588)

In [23]:
bow_test.shape

(17220, 357588)

In [24]:
index = y_train[y_train['grades']!=y_train['grades']].index

In [25]:
bow = bow.iloc[bow.index.drop(index)]
y_train = y_train.drop(index, axis=0)

In [26]:
bow_test.shape, bow.shape

((17220, 357588), (51476, 357588))

In [27]:
from sklearn.linear_model import RidgeClassifier

In [28]:
clf = RidgeClassifier()
clf.fit(bow, y_train)

/Users/aleksandrakozevnikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:1178: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RidgeClassifier()

In [29]:
pred = clf.predict(bow_test)

In [30]:
results = pd.DataFrame(pred, columns =['grades'],dtype=int)

In [31]:
results['inds'] = range(0,len(results))

In [32]:
results.grades.value_counts()

1    11950
5     5041
2      200
3       17
4       12
Name: grades, dtype: int64

In [33]:
results[['inds', 'grades']].to_csv('ridge.csv',index=False)